In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="1"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import subprocess
import matplotlib
matplotlib.use("Agg")
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2

import tensorflow as tf

import shfl

/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/frame

In [2]:
args = {"data_path":"../data/generated/COVIDGR1.0reducido/transformed", 
        "output_path": "../weights",
        "input_path": "",
        "model_name":"transferlearning.model", 
        "label_bin": "lb.pickle", 
        "batch_size": 8,
        "federated_rounds": 1,
        "epochs_per_FL_round": 1,
        "num_nodes": 2,
        "size_averaging": 1,
        "random_rotation": 0,
        "random_shift": 0, 
        "random_zoom": 0,
        "horizontal_flip": False,        
        "finetune": True,
        "train_network": True}

In [3]:
LABELS = set(["NTN", "NTP", "PTN", "PTP"])
print("[INFO] training for labels: " + str(LABELS))

imagePaths = list(paths.list_images(os.path.join(args["data_path"])))
data = []
labels = []
#print(imagePaths)
print("[INFO] loading images...")
for imagePath in imagePaths:
    # extract the class label from the filename
    imgname = imagePath.split(os.path.sep)[-1]
    
    name, class_transf_ext = imgname.split('_')
    class_transf = class_transf_ext.split('.')[0]
    class_label, transf = class_transf.split('T18')
    label = class_label+"T"+transf

    # load the image, convert it to RGB channel ordering, and resize
    # it to be a fixed 224x224 pixels, ignoring aspect ratio
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))

    # update the data and labels lists, respectively
    data.append(image)
    labels.append(label)

# convert the data and labels to NumPy arrays
data = np.array(data)
labels = np.array(labels)

# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
print("[INFO] done")
#print(len(LABELS))


[INFO] training for labels: {'PTN', 'PTP', 'NTP', 'NTN'}
[INFO] loading images...
[INFO] done


In [4]:
#print(labels)

In [11]:
#database = shfl.data_base.LabeledDatabase(data, labels)
database = shfl.data_base.DatabaseFromDirectory("../data/generated/COVIDGR1.0reducido/transformed-split", height = 224, width = 224)
train_data, train_labels, test_data, test_labels = database.load_data()

print("[INFO] Number of train images: " + str(len(train_data)))
print("[INFO] Number of test images: " + str(len(test_data)))

print("[INFO] Distributing the train set across the nodes...")
iid_distribution = shfl.data_distribution.IidDataDistribution(database)
federated_data, test_data, test_label = iid_distribution.get_federated_data(num_nodes=args["num_nodes"])
print("[INFO] done")

[INFO] Number of train images: 28
[INFO] Number of test images: 8
[[0 1 0 0]
 [0 1 0 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 1 0]
 [0 0 1 0]
 [1 0 0 0]
 [1 0 0 0]]
[INFO] Distributing the train set across the nodes...
[INFO] done


In [6]:
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input

"""
datagen_train = ImageDataGenerator(
    preprocessing_function = preprocess_input,
    rotation_range = args["random_rotation"],
    width_shift_range = args["random_shift"],
    height_shift_range = args["random_shift"],
    zoom_range = args["random_zoom"],
    horizontal_flip = args["horizontal_flip"]
)
"""

datagen_train = ImageDataGenerator(preprocessing_function = preprocess_input)
datagen_val = ImageDataGenerator(preprocessing_function = preprocess_input)

class TransferLearningModel(shfl.model.DeepLearningModel):    
    
    def train(self, data, labels):
        #self._check_data(data)
        #self._check_labels(labels)

        #early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='min')
        self._model.fit(
            x=datagen_train.flow(data, labels, batch_size=args["batch_size"]),
            steps_per_epoch=len(data) // args["batch_size"],
            validation_data=datagen_val.flow(test_data, test_labels),
            validation_steps=len(test_data) // args["batch_size"],
            epochs=self._epochs
        )

def model_builder():
    
    resnet50 = tf.keras.applications.ResNet50(include_top = False, weights = 'imagenet', pooling = 'avg', input_tensor=Input(shape=(224, 224, 3)))
    
    if (args["finetune"]):
        resnet50.trainable = False
    else: 
        resnet50.trainable = True
    
    # Add last layers
    x = resnet50.output
    # x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(512, activation = 'relu')(x)
    # x = tf.keras.layers.Dropout(0.5)(x)
    predictions = tf.keras.layers.Dense(len(LABELS), activation = 'softmax')(x)
    
    model = tf.keras.Model(inputs = resnet50.input, outputs = predictions)
    
    criterion = tf.keras.losses.CategoricalCrossentropy()
    optimizer = tf.keras.optimizers.SGD(lr = 1e-3, decay = 1e-6, momentum = 0.9, nesterov = True)
    metrics = [tf.keras.metrics.categorical_accuracy]
    
    return TransferLearningModel(model=model, criterion=criterion, optimizer=optimizer, metrics=metrics)


In [7]:
from tensorflow.keras.models import load_model

# Train the network:
epochs_per_FL_round=args["epochs_per_FL_round"]
aggregator = shfl.federated_aggregator.FedAvgAggregator()
federated_government = shfl.federated_government.FederatedGovernment(model_builder, federated_data, aggregator)
if args["train_network"]:
    federated_government.run_rounds(args["federated_rounds"], test_data, test_label)
    print("[INFO] saving model ...")
    federated_government.global_model._model.save( os.path.join(args["output_path"], args["model_name"]) )
    print("[INFO] done")
else:
    print("[INFO] loading pre-computed model ...")
    model_path = os.path.join(args["output_path"], args["model_name"])
    federated_government.global_model._model = load_model(model_path)
    #lb = pickle.loads(open(os.path.join(args["output_path"], args["label_bin"]), "rb").read())
    print("[INFO] done")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Accuracy round 0
1/1 [==============================] - 4s 4s/step - loss: 1.5286 - categorical_accuracy: 0.2500 - val_loss: 2.3227 - val_categorical_accuracy: 0.2500
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x7fbe76a3fc88>: [2.5782532691955566, 0.25]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x7fbe76a3f4e0>: [2.6029462814331055, 0.25]
Global model test performance : [2.59030818939209, 0.25]



[INFO] saving model ...
[INFO] done


In [8]:
# evaluate the federated network
print("[INFO] evaluating federated network...")
predictions = federated_government.global_model.predict(data=test_data.astype("float32"))

print(classification_report(test_labels.argmax(axis=1), predictions, target_names=lb.classes_) )

[INFO] evaluating federated network...
[1 1 1 1 1 1 1 1]
              precision    recall  f1-score   support

         NTN       0.00      0.00      0.00         2
         NTP       0.25      1.00      0.40         2
         PTN       0.00      0.00      0.00         2
         PTP       0.00      0.00      0.00         2

    accuracy                           0.25         8
   macro avg       0.06      0.25      0.10         8
weighted avg       0.06      0.25      0.10         8



/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
transformed_path = args["data_path"]
sinsegmentar_path = "./data/input/COVIDGR1.0reducido-SinSegmentar"

test_files = os.listdir(args["data_path"])
print(test_data)

[[[[ 26 193 187]
   [ 75  67 206]
   [ 69 232 194]
   ...
   [ 47 221 192]
   [124  86 212]
   [124  69 242]]

  [[ 62 230  79]
   [234  87  39]
   [217 218  40]
   ...
   [120 233  35]
   [142  65  62]
   [226 235 188]]

  [[228 153 212]
   [199  68 112]
   [160 204 158]
   ...
   [ 76 158  82]
   [ 90 112  83]
   [184 228  37]]

  ...

  [[ 63  63 210]
   [ 97  92  60]
   [105  74  60]
   ...
   [158  66  55]
   [166 119  83]
   [147  73 131]]

  [[ 53 179 231]
   [ 45 194 129]
   [ 66 146  21]
   ...
   [190  98 155]
   [193 159  51]
   [190 209 146]]

  [[ 29  23  63]
   [217  36  48]
   [226  34  41]
   ...
   [197 106 176]
   [208 152 196]
   [188  86 182]]]


 [[[ 41 186 166]
   [ 66 225 188]
   [ 61 214 172]
   ...
   [ 59 198 190]
   [123  85 196]
   [ 96  64 249]]

  [[226  52  37]
   [205  71  31]
   [190 207  37]
   ...
   [201 177  84]
   [196 230  37]
   [220 221 195]]

  [[220 155  78]
   [197  59 103]
   [176 182 150]
   ...
   [210 174 136]
   [226 217  73]
   [211 206